In [1]:
#Importing libraries
import pandas as pd
import numpy as np

In [2]:
#raw data
raw=pd.read_excel(r'C:\Users\Asus\Desktop\raw_data.xlsx') 

In [3]:
# # raw column names
# for col in raw.columns:
#     print(col)
# returns:
# نام
# نام خانوادگی
# کد ملی
# موبایل
# مرحله ثبت نام
# شهر مورد تقاضا
# شهر محل سکونت
# آدرس متقاضی
# کدپستی
# وضعیت بارگذاری مدارک
# وضعیت سرپرستی
# شغل
# جنسیت
# میزان تحصیلات
# نام پدر
# شماره شناسنامه
# تاریخ تولد
# تعداد فرزندان
# بعد خانوار
# کد شعبه
# نام شعبه
# شماره حساب


In [4]:
#useful features:

# نام
# نام خانوادگی
#   کد ملی
# شماره شناسنامه
# شهر مورد تقاضا
# شهر محل سکونت
# کدپستی

raw=raw.loc[:, ["نام",'نام خانوادگی',"شماره شناسنامه","کد ملی","شهر محل سکونت",'شهر مورد تقاضا','کدپستی']]

In [5]:
#renaming
raw.rename(columns={'نام':"first_name" ,  "نام خانوادگی":"last_name"  ,  "کد ملی":"codemeli"  ,"شماره شناسنامه":"shomareh_shenasnameh", 
                    "کدپستی": "postal_code" ,"شهر مورد تقاضا"  : "city_requested " , "شهر محل سکونت" :"city_residance"} , inplace=True)

raw

,first_name,last_name,shomareh_shenasnameh,codemeli,city_residance,city_requested,postal_code
0,حسن,گروسی,4442,5098674451,بوئین زهرا,بوئین زهرا,3454178378
1,مرتضی,حاجی حسینی,1485,4390754092,بوئین زهرا,بوئین زهرا,3451738961
2,رحمان,رفیعی,13,4391615207,بوئین زهرا,بوئین زهرا,3451646436
3,ابوالفضل,احمدی,634,5098799899,بوئین زهرا,بوئین زهرا,3451737955
4,جعفر,برزگراحمدی,373,5099580238,بوئین زهرا,بوئین زهرا,3451637311
...,...,...,...,...,...,...,...
6894,صادق,صادقی راد,4380317031,4380317031,قزوین,قزوین,NaN
6895,بهمن,گنج ها,4310661378,4310661378,قزوین,قزوین,3431164685
6896,زهرا,امانی,4488,4324466017,قزوین,قزوین,3416664781
6897,مجید,عسگری,512,4323123175,قزوین,قزوین,3471679831


In [6]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6899 entries, 0 to 6898
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   first_name            6899 non-null   object
 1   last_name             6899 non-null   object
 2   shomareh_shenasnameh  6899 non-null   int64 
 3   codemeli              6899 non-null   int64 
 4   city_residance        6899 non-null   object
 5   city_requested        6899 non-null   object
 6   postal_code           6214 non-null   object
dtypes: int64(2), object(5)
memory usage: 377.4+ KB


#codemeli

In [7]:
# digits counting

def count_digits(string):
    
    return sum(item.isdigit() for item in string)

In [8]:
#shomaresh argham codemeli 

raw['Tedad_argham_codemeli'] = (raw['codemeli'].astype(str)).apply(count_digits)
raw[['Tedad_argham_codemeli']].value_counts() 

Tedad_argham_codemeli
10                       6581
8                         188
9                         130
dtype: int64

در مرحله اول باید بررسی بر روی تعداد کاراکترها را انجام دهید، از آنجایی که در سیستم کد ملی معمولاً قبل از کد تعدادی صفر وجود دارد (رقم اول و رقم دوم از سمت چپ کد ملی ممکن است صفر باشد) و در بسیاری از موارد ممکن است کاربر این صفرها را وارد نکرده باشد و یا نرم افزار این صفرها را ذخیره نکرده باشد بهتر است قبل از هر کاری در صورتی که طول کد بزرگ‌تر مساوی ۸ و کمتر از ۱۰ بود به تعداد لازم (یک تا دو تا صفر) به سمت چپ عدد اضافه کنید.

https://exceliran.com/melli-code-checker-in-excel/  : منبع

In [9]:
list_0=[]


for i in raw['codemeli'].astype(str):
    if count_digits(i)==8:
        i='00'+i
        #i = int(i) 
        list_0.append(i)
    elif count_digits(i)==9:
        i='0'+i
        list_0.append(i)
    else:
        list_0.append(i)     


list_0=(pd.DataFrame(list_0 , columns=['codemeli']))
raw['codemeli'] = list_0.values #type: str


In [10]:
raw.drop(columns=[ 'Tedad_argham_codemeli'] , inplace=True)

 کدهای ملی که همه ارقام آنها مثل هم باشند معتبر نیستند.

In [11]:
invalid_list= ['0000000000','1111111111','2222222222','3333333333','4444444444','5555555555','6666666666','7777777777','8888888888','9999999999',]
for i in raw['codemeli']:

    if i in invalid_list:
        print(i) #returns none
    
            
    


#raw[raw.codemeli == '']


در مرحله دوم بعد از اطمینان از ۱۰ رقم بودن شماره ملی، ضرایب ۲ الی ۱۰ را در ۹ رقم سمت چپ شماره ملی نظیر به نظیر ضرب کرده و سپس تمامی اعداد را جمع می‌کنیم.
مجموع بدست آمده از مرحله دو را بر عدد ۱۱ تقسیم می‌کنیم و باقیمانده آن را محاسبه می‌کنیم.

باقیمانده تقسیم برابر ۵ می‌باشد.



حالت اول:

اگر باقیمانده کمتر از ۲ باشد، رقم کنترل ( رقم سمت راست شماره ملی) باید برابر باقیمانده این تقسیم باشد. یعنی یا صفر و یا یک، که در این مثال ۵ می‌باشد و شامل حالت اول نمی‌گردد پس باید حالت دوم را مد نظر قرار داد.


حالت دوم:

اگر عدد باقیمانده تقسیم بزرگتر مساوی ۲ باشد عدد ۱۱ را منهای باقیمانده تقسیم مرحله قبل می‌کنیم، حال باید عدد حاصل شده برابر رقم کنترل(رقم سمت راست کد ملی) باشد.

https://exceliran.com/melli-code-checker-in-excel/  : منبع

In [12]:
list_1=[]

for codemeli in (raw['codemeli']):
    
    Sum=0
    
    for j in range(9):
        
        Sum+= int(codemeli[j])*(10 - j) #sum number code meli 1-9
        
    C=(Sum%11)
    
    A=int(codemeli[9]) #ragham e control
    
    B=11-C   
    
    if ((C<2 and A==0 )) or (C<2 and A==1):
        
        list_1.append(codemeli[:3])
        
    elif (C>=2) and (B==A):
        
        list_1.append(codemeli[:3])
    else:
        list_1.append("999")
        
        
    
list_1=pd.DataFrame(list_1,columns =['list_1'])
# list_1[list_1.list_1 == '999']  returns zero




In [13]:
raw['se_ragham_aval_e_codemeli'] = list_1.values #type: str


In [15]:
#importing new dataset 
cm_shahr_dataset=pd.read_excel(r'C:\Users\Asus\Desktop\codemeli_shahr_ir.xlsx') 
#converting  to dict
key, value = list(cm_shahr_dataset['she_ragham_aval'].astype(str)), list(cm_shahr_dataset['shahr'])
cm_shahr_dataset = dict(zip(key, value))    

In [16]:
 list_2= []

for i in raw['se_ragham_aval_e_codemeli']:
    
    if i in key:
        
        list_2.append(cm_shahr_dataset[i])
    else:
        list_2.append("null")

In [17]:
list_2=pd.DataFrame ( list_2 , columns=['list_2'])
raw['shahr_e_sodor_shenasnameh']=list_2.values

In [ ]:
raw

,first_name,last_name,shomareh_shenasnameh,codemeli,city_residance,city_requested,postal_code,se_ragham_aval_e_codemeli,shahr_e_sodor_shenasnameh
0,حسن,گروسی,4442,5098674451,بوئین زهرا,بوئین زهرا,3454178378,509,بوئین زهرا
1,مرتضی,حاجی حسینی,1485,4390754092,بوئین زهرا,بوئین زهرا,3451738961,439,تاکستان
2,رحمان,رفیعی,13,4391615207,بوئین زهرا,بوئین زهرا,3451646436,439,تاکستان
3,ابوالفضل,احمدی,634,5098799899,بوئین زهرا,بوئین زهرا,3451737955,509,بوئین زهرا
4,جعفر,برزگراحمدی,373,5099580238,بوئین زهرا,بوئین زهرا,3451637311,509,بوئین زهرا
...,...,...,...,...,...,...,...,...,...
6894,صادق,صادقی راد,4380317031,4380317031,قزوین,قزوین,NaN,438,تاکستان
6895,بهمن,گنج ها,4310661378,4310661378,قزوین,قزوین,3431164685,431,قزوین
6896,زهرا,امانی,4488,4324466017,قزوین,قزوین,3416664781,432,قزوین
6897,مجید,عسگری,512,4323123175,قزوین,قزوین,3471679831,432,قزوین


validation of postal_code

الگوريتم صحت سنجي عددي كد پستي:

- تعداد ارقام كد پستي 10 رقم مي باشد.

- عدد صفر در كد پستي استفاده نشده است.

- عدد 2 در كد پستي استفاده نشده است.

- چهار رقم ابتدايي نمي تواند شبيه هم باشد.

- رقم پنجم نمي تواند عدد 5 باشد.

-تمامي ارقام نمي تواند شبيه هم باشد.

In [28]:
raw[['postal_code']].info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6899 entries, 0 to 6898
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   postal_code  6214 non-null   object
dtypes: object(1)
memory usage: 54.0+ KB


In [36]:
raw[['postal_code']].isna().sum()  #returns 685

#replacing nans with 1111111111
raw[['postal_code']]=raw[['postal_code']].fillna(1111111111)

# raw[['postal_code']].isna().sum()#returns 0 #double check

In [37]:
raw[['postal_code']].value_counts()

postal_code
1111111111     685
1111111111     123
3414739866      39
3481634816      28
3451634516      22
              ... 
3419944347       1
3419944164       1
3419943643       1
3419943554       1
3447131117       1
Length: 5636, dtype: int64

# تعداد ارقام كد پستي 10 رقم مي باشد


In [38]:
raw['digits_counting_postal_code'] = (raw['postal_code'].astype(str)).apply(count_digits)
raw['digits_counting_postal_code'].value_counts()

10    6879
9       11
8        3
5        2
0        2
2        1
6        1
Name: digits_counting_postal_code, dtype: int64

In [41]:
list_3=[]

for i in raw['postal_code'].astype(str):
    
    
    if count_digits(i)==10:
        
            
        list_3.append(i)
        
    else:
         list_3.append('1111111111')
            
list_3= pd.DataFrame(list_3 , columns=['list_3']) #list_3.shape returns 6899 rows × 1 columns 
list_3.value_counts()  #returns 828 ta '1111111111' 


list_3    
1111111111    828
3414739866     39
3481634816     28
3451634516     22
3419113131     20
             ... 
3419964497      1
3419964444      1
3419963414      1
3419958931      1
3447144197      1
Length: 5620, dtype: int64

In [42]:
 #replacing raw['postal_code'] with list_3

raw['postal_code']= list_3['list_3'].values

# - عدد صفر در كد پستي استفاده نشده است

# - عدد 2 در كد پستي استفاده نشده است

In [43]:
list_4=[]

for i in raw['postal_code'].astype(str):
    
    if i.count('2') == 0  and i.count('0')==0:
        
        list_4.append(i)
        
    elif i.count('2') != 0  or i.count('0')!=0:
        
        list_4.append('1111111111')
        
        
        
list_4= pd.DataFrame(list_4 , columns=['list_4'])
list_4.value_counts() #returns 1288 ta '1111111111' 

list_4    
1111111111    1288
3414739866      39
3481634816      28
3451634516      22
3419113131      20
              ... 
3419937688       1
3419937541       1
3419937457       1
3419937387       1
3445114151       1
Length: 5198, dtype: int64

In [44]:
# replacing raw['postal_code'] (list_3) with list_4

raw['postal_code']= list_4['list_4'].values

# - چهار رقم ابتدايي نمي تواند شبيه هم باشد


In [45]:
list_5=[]

for i in raw['postal_code'].astype(str):
    
    if i[0:1 ]!= i[1:2]!= i[2:3]!= i[3:4] :
        
        list_5.append(i)
    else:
        list_5.append('1111111111')
        
list_5= pd.DataFrame(list_5 , columns=['list_5'])
list_5.value_counts() #returns 1971 ta '1111111111' 


list_5    
1111111111    1971
3414739866      39
3481634816      28
3451634516      22
3419113131      20
              ... 
3419635358       1
3419635459       1
3419645714       1
3419645984       1
9861437513       1
Length: 4563, dtype: int64

In [52]:
# replacing raw['postal_code'] (list_4) with list_5

raw['postal_code']= list_5['list_5'].values

# - رقم پنجم نمي تواند عدد 5 باشد


In [54]:
list_6=[]

for i in raw['postal_code'].astype(str):
    
    if i[4:5]!='5':
        
        list_6.append(i)
    else:
        list_6.append('1111111111')
        
list_6= pd.DataFrame(list_6 , columns=['list_6'])
list_6.value_counts() #returns 1971 ta '1111111111' 


list_6    
1111111111    1971
3414739866      39
3481634816      28
3451634516      22
3419113131      20
              ... 
3419635358       1
3419635459       1
3419645714       1
3419645984       1
9861437513       1
Length: 4563, dtype: int64

In [55]:
raw['postal_code']= list_6['list_6'].values

# -تمامي ارقام نمي تواند شبيه هم باشد

In [60]:
list_7=[]

invalid_list=["0000000000","1111111111","2222222222","3333333333","4444444444","5555555555","6666666666","7777777777"
              ,"8888888888","9999999999"]



for i in raw['postal_code'].astype(str):
    
    
    if i in invalid_list:
        
            
        list_7.append('1111111111')
        
    else:
         list_7.append(i)
            
list_7= pd.DataFrame(list_7 , columns=['list_7'])

list_7.value_counts() #returns 1971 ta '1111111111' 

raw['postal_code']= list_7['list_7'].values

In [61]:
panj_ragham_aval_postal_code=[]
for postal_code in raw['postal_code'].astype(str):
    panj_ragham_aval_postal_code.append(postal_code[:5])
panj_ragham_aval_postal_code=pd.DataFrame(panj_ragham_aval_postal_code,columns=['panj_ragham_aval_postal_code'])
raw['panj_ragham_aval_postal_code']=panj_ragham_aval_postal_code.values

In [63]:
raw[['panj_ragham_aval_postal_code']].drop_duplicates()

,panj_ragham_aval_postal_code
0,34541
1,34517
2,34516
5,34519
10,34518
...,...
6163,31956
6227,43157
6249,43199
6344,18968


In [65]:
#importing pc
pc_df=pd.read_excel(r'C:\Users\Asus\Desktop\maskan_mehr_project\pc.xlsx') 
pc_df.drop(columns=[ 'Unnamed: 0'] , inplace=True)
pc_df


,pc,city,latitude(φ),longitude(λ),φ(d),λ(d)
0,11111,NaN,NaN,NaN,NaN,NaN
1,31319,NaN,NaN,NaN,NaN,NaN
2,31956,NaN,NaN,NaN,NaN,NaN
3,43131,NaN,NaN,NaN,NaN,NaN
4,43141,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
158,34917,محمدیه,"36° 13' 26.465"" N","50° 10' 57.313"" E",36.224018,50.182587
159,34918,محمدیه,"36° 13' 26.465"" N","50° 10' 57.313"" E",36.224018,50.182587
160,34919,محمدیه,"36° 13' 26.465"" N","50° 10' 57.313"" E",36.224018,50.182587
161,34931,معلم کلایه,"36° 26' 50.197"" N","50° 28' 41.506"" E",36.447277,50.478195


In [66]:
#converting pc_df  to dict
keyy, valuee = list(pc_df['pc']), list(pc_df['city'])
dict2 = dict(zip(keyy, valuee))

In [70]:
code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

city_2 = []

for i in code2:
    
    if i in keyy:
        
        city_2.append(dict2[i])
    else:
        city_2.append("null")

raw['shahr_e_code_posti']=pd.DataFrame(city_2).values



,first_name,last_name,shomareh_shenasnameh,codemeli,city_residance,city_requested,postal_code,se_ragham_aval_e_codemeli,shahr_e_sodor_shenasnameh,digits_counting_postal_code,panj_ragham_aval_postal_code,shahr_e_code_posti
0,حسن,گروسی,4442,5098674451,بوئین زهرا,بوئین زهرا,3454178378,509,بوئین زهرا,10,34541,بوئین زهرا
1,مرتضی,حاجی حسینی,1485,4390754092,بوئین زهرا,بوئین زهرا,3451738961,439,تاکستان,10,34517,بوئین زهرا
2,رحمان,رفیعی,13,4391615207,بوئین زهرا,بوئین زهرا,3451646436,439,تاکستان,10,34516,بوئین زهرا
3,ابوالفضل,احمدی,634,5098799899,بوئین زهرا,بوئین زهرا,3451737955,509,بوئین زهرا,10,34517,بوئین زهرا
4,جعفر,برزگراحمدی,373,5099580238,بوئین زهرا,بوئین زهرا,3451637311,509,بوئین زهرا,10,34516,بوئین زهرا
...,...,...,...,...,...,...,...,...,...,...,...,...
6894,صادق,صادقی راد,4380317031,4380317031,قزوین,قزوین,1111111111,438,تاکستان,10,11111,NaN
6895,بهمن,گنج ها,4310661378,4310661378,قزوین,قزوین,3431164685,431,قزوین,10,34311,الوند
6896,زهرا,امانی,4488,4324466017,قزوین,قزوین,3416664781,432,قزوین,10,34166,قزوین
6897,مجید,عسگری,512,4323123175,قزوین,قزوین,3471679831,432,قزوین,10,34716,تاکستان


In [72]:
# city_cr_pc
raw['city_cr_pc'] = np.where(  (raw['city_residance'] == raw['shahr_e_code_posti']) , 'matched' ,'not_matched' )
raw[['city_cr_pc','city_residance','shahr_e_code_posti']]

,city_cr_pc,city_residance,shahr_e_code_posti
0,matched,بوئین زهرا,بوئین زهرا
1,matched,بوئین زهرا,بوئین زهرا
2,matched,بوئین زهرا,بوئین زهرا
3,matched,بوئین زهرا,بوئین زهرا
4,matched,بوئین زهرا,بوئین زهرا
...,...,...,...
6894,not_matched,قزوین,NaN
6895,not_matched,قزوین,الوند
6896,matched,قزوین,قزوین
6897,not_matched,قزوین,تاکستان


In [73]:
# latitude(φ)	


key_1, value_1 = list(pc_df['pc']), list(pc_df['latitude(φ)'])
dict_lat = dict(zip(key_1, value_1))

# dict_lat


code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

lat = []

for i in code2:
    
    if i in key_1:
        
        lat.append(dict_lat[i])
    else:
        lat.append("null")

raw['latitude(φ)_by_postal_code']=pd.DataFrame(lat).values

In [74]:
# latitude(φ)	


key_1, value_1 = list(pc_df['pc']), list(pc_df['latitude(φ)'])
dict_lat = dict(zip(key_1, value_1))

# dict_lat


code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

lat = []

for i in code2:
    
    if i in key_1:
        
        lat.append(dict_lat[i])
    else:
        lat.append("null")

raw['latitude(φ)_by_postal_code']=pd.DataFrame(lat).values

In [75]:
# longitude(λ)

key_2, value_2 = list(pc_df['pc']), list(pc_df['longitude(λ)'])
dict_longt = dict(zip(key_2, value_2))



code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

longt = []

for i in code2:
    
    if i in key_2:
        
        longt.append(dict_longt[i])
    else:
        longt.append("null")

raw['longitude(λ)_by_postal_code']=pd.DataFrame(longt).values

In [76]:
# φ(d)

key_3, value_3 = list(pc_df['pc']), list(pc_df['φ(d)'])
dict_φ = dict(zip(key_3, value_3))

# dict_φ


code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

φ = []

for i in code2:
    
    if i in key_3:
        
        φ.append(dict_φ[i])
    else:
        φ.append("null")

raw['φ(d)_by_postal_code']=pd.DataFrame(φ).values

In [77]:
# λ(d)

key_4, value_4 = list(pc_df['pc']), list(pc_df[' λ(d)'])

dict_λ = dict(zip(key_4, value_4))

# dict_λ


code2 = list(raw['panj_ragham_aval_postal_code'].astype(int))

λ = []

for i in code2:
    
    if i in key_4:
        
        λ.append(dict_λ[i])
    else:
        λ.append("null")

raw['λ(d)_by_postal_code']=pd.DataFrame(λ).values

In [78]:
raw

,first_name,last_name,shomareh_shenasnameh,codemeli,city_residance,city_requested,postal_code,se_ragham_aval_e_codemeli,shahr_e_sodor_shenasnameh,digits_counting_postal_code,panj_ragham_aval_postal_code,shahr_e_code_posti,city_cr_pc,latitude(φ)_by_postal_code,longitude(λ)_by_postal_code,φ(d)_by_postal_code,λ(d)_by_postal_code
0,حسن,گروسی,4442,5098674451,بوئین زهرا,بوئین زهرا,3454178378,509,بوئین زهرا,10,34541,بوئین زهرا,matched,"35° 45' 25.744"" N","50° 3' 43.085"" E",35.757153,50.06197
1,مرتضی,حاجی حسینی,1485,4390754092,بوئین زهرا,بوئین زهرا,3451738961,439,تاکستان,10,34517,بوئین زهرا,matched,"35° 45' 25.744"" N","50° 3' 43.085"" E",35.757153,50.06197
2,رحمان,رفیعی,13,4391615207,بوئین زهرا,بوئین زهرا,3451646436,439,تاکستان,10,34516,بوئین زهرا,matched,"35° 45' 25.744"" N","50° 3' 43.085"" E",35.757153,50.06197
3,ابوالفضل,احمدی,634,5098799899,بوئین زهرا,بوئین زهرا,3451737955,509,بوئین زهرا,10,34517,بوئین زهرا,matched,"35° 45' 25.744"" N","50° 3' 43.085"" E",35.757153,50.06197
4,جعفر,برزگراحمدی,373,5099580238,بوئین زهرا,بوئین زهرا,3451637311,509,بوئین زهرا,10,34516,بوئین زهرا,matched,"35° 45' 25.744"" N","50° 3' 43.085"" E",35.757153,50.06197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6894,صادق,صادقی راد,4380317031,4380317031,قزوین,قزوین,1111111111,438,تاکستان,10,11111,NaN,not_matched,NaN,NaN,NaN,NaN
6895,بهمن,گنج ها,4310661378,4310661378,قزوین,قزوین,3431164685,431,قزوین,10,34311,الوند,not_matched,"36° 10' 13.224"" N","50° 4' 26.958"" E",36.170341,50.074154
6896,زهرا,امانی,4488,4324466017,قزوین,قزوین,3416664781,432,قزوین,10,34166,قزوین,matched,"36° 16' 54.282"" N","50° 0' 7.085"" E",36.281746,50.001968
6897,مجید,عسگری,512,4323123175,قزوین,قزوین,3471679831,432,قزوین,10,34716,تاکستان,not_matched,"36° 4' 17.112"" N","49° 41' 46.176"" E",36.071419,49.696159
